### Pandas Lab: Time Shifts & Multi Level Indexing

This lab is designed to introduce you to working with time in a more granular way, and understanding how to build features when your data has hierarchies or panels.  

Ie, when you have repeated observations for the same objects.  This is an important concept because lots of statistical methods don't explicitly account for values which might naturally be correlated with one another over time.  

But lots of data **is** highly correlated over time!  

By the time you're done with this lab, you'll have built 9 columns that capture a variety of information about how an observed value is changing with respect to itself.

**Question 1:** Set the multi-level index so the first level is the Stock symbol itself, and the second level is the date.  Make sure the date column is sorted in ascending order.  You might have to use the `sort_index(level=0)` method to get the values straight.

In [7]:
import pandas as pd
df = pd.read_csv('../../data/stocks_panel.csv', parse_dates=['Date'])

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6285 entries, 0 to 6284
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    6285 non-null   datetime64[ns]
 1   Stock   6285 non-null   object        
 2   Price   6285 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 147.4+ KB


In [11]:
df = df.set_index(['Stock', 'Date']).sort_index(level=0)
df.head()

Price
Stock Date                  
AAPL  2014-11-05  108.860001
      2014-11-06  108.699997
      2014-11-07  109.010002
      2014-11-10  108.830002
      2014-11-11  109.699997

**Question 2:** To capture some other aspects of dates, create columns in your dataset that capture this aspect of each timestamp:

  - What quarter it's in
  - Whether or not it's the last day of the month/quarter
  - What day it is (ie, do price changes vary by day?)
  
**Hint:** You don't use the `dt` attribute to get date parts from index values.  Multi indices are also a little tricky.  

To get what you want, try this: `df.index.get_level_values(level=1).your_datepart_here`

In [17]:
dates = df.index.get_level_values(level=1)
df['Quarter'] = dates.quarter
df['MonthEnd'] = dates.is_month_end
df['QuarterEnd'] = dates.is_quarter_end
df['DayOfWeek'] = dates.dayofweek
df

Price  Quarter  MonthEnd  QuarterEnd  DayOfWeek
Stock Date                                                            
AAPL  2014-11-05  108.860001        4     False       False          2
      2014-11-06  108.699997        4     False       False          3
      2014-11-07  109.010002        4     False       False          4
      2014-11-10  108.830002        4     False       False          0
      2014-11-11  109.699997        4     False       False          1
...                      ...      ...       ...         ...        ...
MSFT  2019-10-28  144.190002        4     False       False          0
      2019-10-29  142.830002        4     False       False          1
      2019-10-30  144.610001        4     False       False          2
      2019-10-31  143.369995        4      True       False          3
      2019-11-01  143.720001        4     False       False          4

[6285 rows x 5 columns]

**Question 3:** Time Series Embedding

Lots of times if you're trying to predict the value of something tomorrow, the most import piece of information is what the value of something is today, and yesterday, and so on.

Try and create columns that capture previously observed values for each stock.  

Make two columns that capture the value of the following:

 - What the previous recorded price for each stock was
 - The stock price from two observations ago
 
**Remember:** This has to be done on a particular level of the index to make sure it's getting applied appropriately!

In [36]:
df['Prev. Price'] = df.groupby(level=0)['Price'].shift()
df['2day Prior Price'] = df.groupby(level=0)['Price'].shift(2)
df

Price  Quarter  MonthEnd  QuarterEnd  DayOfWeek  \
Stock Date                                                               
AAPL  2014-11-05  108.860001        4     False       False          2   
      2014-11-06  108.699997        4     False       False          3   
      2014-11-07  109.010002        4     False       False          4   
      2014-11-10  108.830002        4     False       False          0   
      2014-11-11  109.699997        4     False       False          1   
...                      ...      ...       ...         ...        ...   
MSFT  2019-10-28  144.190002        4     False       False          0   
      2019-10-29  142.830002        4     False       False          1   
      2019-10-30  144.610001        4     False       False          2   
      2019-10-31  143.369995        4      True       False          3   
      2019-11-01  143.720001        4     False       False          4   

                  Prev. Price  2day Prior Price  
Stock Date                                       
AAPL  2014-11-05          NaN               NaN  
      2014-11-06   108.860001               NaN  
      2014-11-07   108.699997        108.860001  
      2014-11-10   109.010002        108.699997  
      2014-11-11   108.830002        109.010002  
...                       ...               ...  
MSFT  2019-10-28   140.729996        139.940002  
      2019-10-29   144.190002        140.729996  
      2019-10-30   142.830002        144.190002  
      2019-10-31   144.610001        142.830002  
      2019-11-01   143.369995        144.610001  

[6285 rows x 7 columns]

**Question 4:** How did each stock price change compared to the S&P 500? 

Lots of times it's useful to see how something else moves with some other item that you're trying to track.  

In the data folder is a file called `s&p.csv`, and it contains the price history of the S&P 500 index for each day since its inception. See if you can upload it, and merge the `adj close` column into your dataset, so there's a column that displays the observed value of the index for every single price observation we have in our dataset.

**Hints:**
 - Merging on multi-level indices is tricky and prone to failure.  To make this a little bit easier, just use `reset_index()` to pop out the date column in the multi-index, and merge on it as if it were a regular column.
 - Make sure both date columns are actually encoded as dates, rather than strings, or else the merge won't work.
 - You'll want to go back to the multi-level index when you're done with this step.

In [38]:
snp = pd.read_csv('../../data/s&p.csv', parse_dates=['Date'])
snp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23138 entries, 0 to 23137
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Date       23138 non-null  datetime64[ns]
 1   Open       23138 non-null  float64       
 2   High       23138 non-null  float64       
 3   Low        23138 non-null  float64       
 4   Close      23138 non-null  float64       
 5   Adj Close  23138 non-null  float64       
 6   Volume     23138 non-null  int64         
dtypes: datetime64[ns](1), float64(5), int64(1)
memory usage: 1.2 MB


In [42]:
df = df.reset_index().merge(snp[['Date', 'Adj Close']], left_on='Date', right_on='Date', how='left').set_index(['Stock', 'Date'])
df

Price  Quarter  MonthEnd  QuarterEnd  DayOfWeek  \
Stock Date                                                               
AAPL  2014-11-05  108.860001        4     False       False          2   
      2014-11-06  108.699997        4     False       False          3   
      2014-11-07  109.010002        4     False       False          4   
      2014-11-10  108.830002        4     False       False          0   
      2014-11-11  109.699997        4     False       False          1   
...                      ...      ...       ...         ...        ...   
MSFT  2019-10-28  144.190002        4     False       False          0   
      2019-10-29  142.830002        4     False       False          1   
      2019-10-30  144.610001        4     False       False          2   
      2019-10-31  143.369995        4      True       False          3   
      2019-11-01  143.720001        4     False       False          4   

                  Prev. Price  2day Prior Price    Adj Close  
Stock Date                                                    
AAPL  2014-11-05          NaN               NaN  2023.569946  
      2014-11-06   108.860001               NaN  2031.209961  
      2014-11-07   108.699997        108.860001  2031.920044  
      2014-11-10   109.010002        108.699997  2038.260010  
      2014-11-11   108.830002        109.010002  2039.680054  
...                       ...               ...          ...  
MSFT  2019-10-28   140.729996        139.940002  3039.419922  
      2019-10-29   144.190002        140.729996  3036.889893  
      2019-10-30   142.830002        144.190002  3046.770020  
      2019-10-31   144.610001        142.830002  3037.560059  
      2019-11-01   143.369995        144.610001  3066.909912  

[6285 rows x 8 columns]

**Question 5:** Window Statistics

Lots of times, if we want to capture some idea of momentum, or how some value changes with what's usually observed.

Ie, if we had 48 purchases in a store today, how does that number compare to what's happened in the last 14 days?  Are things trending up or trending down?  

This also allows us to get a clearer picture of general trends in values, even if there are irregular daily spikes.

To handle these sorts of issues, pandas has an entire section to calculate window statistics called `rolling`, it works like this:

In [47]:
# I'll create a sample dataframe with 30 days worth of values
import numpy as np
index = pd.date_range(start='01/01/2020', end='02/05/2020')
sample_df = pd.DataFrame(np.random.randn(36), index=index, columns=['Value'])
# and here's what it looks like
sample_df.shape

(36, 1)

In [48]:
# and now we'll see rolling 10 day averages
sample_df.rolling(10).mean()

,Value
2020-01-01,NaN
2020-01-02,NaN
2020-01-03,NaN
2020-01-04,NaN
2020-01-05,NaN
2020-01-06,NaN
2020-01-07,NaN
2020-01-08,NaN
2020-01-09,NaN
2020-01-10,-0.244279


You can specify the number of observations to calculate, and choose your aggregator -- `mean()`, `min()`, `sum()`, etc, although `mean()` is the most common.

**Your Turn:** Calculate the rolling 5 & 10 day moving averages for each stock inside the dataset.

**Note:** Do *not* try and merge them back into your dataset yet, just make sure you have the values showing up.

In [59]:
df.groupby(level=0).rolling(5)['Price'].mean(level=0)

Stock  Stock  Date      
AAPL   AAPL   2014-11-05           NaN
              2014-11-06           NaN
              2014-11-07           NaN
              2014-11-10           NaN
              2014-11-11    109.020000
                               ...    
MSFT   MSFT   2019-10-28    139.694000
              2019-10-29    140.986001
              2019-10-30    142.460001
              2019-10-31    143.145999
              2019-11-01    143.744000
Name: Price, Length: 6285, dtype: float64

In [63]:
df.groupby(level=0).rolling(10)['Price'].mean(level=0).droplevel(0)

Stock  Date      
AAPL   2014-11-05           NaN
       2014-11-06           NaN
       2014-11-07           NaN
       2014-11-10           NaN
       2014-11-11           NaN
                        ...    
MSFT   2019-10-28    139.598001
       2019-10-29    139.724000
       2019-10-30    140.144000
       2019-10-31    140.511999
       2019-11-01    141.142999
Name: Price, Length: 6285, dtype: float64

If you take a look at the index, you should notice that it has *three* levels to it, and not just two like before.  

Combining datasets with differing numbers of levels is cumbersome, and there's a decent amount of churn in what methods work from one version of Pandas to another.  

For now, try and get these values back into your original dataset by taking the following steps:

 - calculate the 5 & 10 rolling averages for each stock price on the multilevel index, and save these as variables, and then use the *values* attribute for each one to drop the index and just get the column values (ask me about this if you have questions)
 - use reset_index() to unstack the index on your original dataframe
 - create new columns for the 5 & 10 day moving averages in the original dataset, using the values from the first step.
 
So as a quick example, it would sort of work like this:

`five_day = df.groupby(level=0)['Price'].your_stuff_here.values`

And then use this as the basis to make your new column from your original dataframe with the reset index.

In [73]:
ten_day = df.groupby(level=0).rolling(10)['Price'].mean(level=0)
df['10dayvals'] = ten_day.values
df['10dayvals']

Stock  Date      
AAPL   2014-11-05           NaN
       2014-11-06           NaN
       2014-11-07           NaN
       2014-11-10           NaN
       2014-11-11           NaN
                        ...    
MSFT   2019-10-28    139.598001
       2019-10-29    139.724000
       2019-10-30    140.144000
       2019-10-31    140.511999
       2019-11-01    141.142999
Name: 10dayvals, Length: 6285, dtype: float64

In [77]:
df2 = df.drop('10dayvals', axis=1).copy() 
df2['10dayvals'] = ten_day.droplevel(0)
df2

Price  Quarter  MonthEnd  QuarterEnd  DayOfWeek  \
Stock Date                                                               
AAPL  2014-11-05  108.860001        4     False       False          2   
      2014-11-06  108.699997        4     False       False          3   
      2014-11-07  109.010002        4     False       False          4   
      2014-11-10  108.830002        4     False       False          0   
      2014-11-11  109.699997        4     False       False          1   
...                      ...      ...       ...         ...        ...   
MSFT  2019-10-28  144.190002        4     False       False          0   
      2019-10-29  142.830002        4     False       False          1   
      2019-10-30  144.610001        4     False       False          2   
      2019-10-31  143.369995        4      True       False          3   
      2019-11-01  143.720001        4     False       False          4   

                  Prev. Price  2day Prior Price    Adj Close   10dayvals  
Stock Date                                                                
AAPL  2014-11-05          NaN               NaN  2023.569946         NaN  
      2014-11-06   108.860001               NaN  2031.209961         NaN  
      2014-11-07   108.699997        108.860001  2031.920044         NaN  
      2014-11-10   109.010002        108.699997  2038.260010         NaN  
      2014-11-11   108.830002        109.010002  2039.680054         NaN  
...                       ...               ...          ...         ...  
MSFT  2019-10-28   140.729996        139.940002  3039.419922  139.598001  
      2019-10-29   144.190002        140.729996  3036.889893  139.724000  
      2019-10-30   142.830002        144.190002  3046.770020  140.144000  
      2019-10-31   144.610001        142.830002  3037.560059  140.511999  
      2019-11-01   143.369995        144.610001  3066.909912  141.142999  

[6285 rows x 9 columns]

In [80]:
(df2['10dayvals'].dropna() == df['10dayvals'].dropna()).value_counts()

True    6240
Name: 10dayvals, dtype: int64

,index,Stock,Date,Price,Quarter,MonthEnd,QuarterEnd,DayOfWeek,Prev. Price,2day Prior Price,Adj Close,10dayvals
0,0,AAPL,2014-11-05,108.860001,4,False,False,2,NaN,NaN,2023.569946,NaN
1,1,AAPL,2014-11-06,108.699997,4,False,False,3,108.860001,NaN,2031.209961,NaN
2,2,AAPL,2014-11-07,109.010002,4,False,False,4,108.699997,108.860001,2031.920044,NaN
3,3,AAPL,2014-11-10,108.830002,4,False,False,0,109.010002,108.699997,2038.260010,NaN
4,4,AAPL,2014-11-11,109.699997,4,False,False,1,108.830002,109.010002,2039.680054,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
6280,6280,MSFT,2019-10-28,144.190002,4,False,False,0,140.729996,139.940002,3039.419922,NaN
6281,6281,MSFT,2019-10-29,142.830002,4,False,False,1,144.190002,140.729996,3036.889893,NaN
6282,6282,MSFT,2019-10-30,144.610001,4,False,False,2,142.830002,144.190002,3046.770020,NaN
6283,6283,MSFT,2019-10-31,143.369995,4,True,False,3,144.610001,142.830002,3037.560059,NaN


In [2]:
import pandas as pd
stocks2 = pd.read_csv('../../data/stocks.csv')

In [4]:
?stocks2.melt

Signature:
stocks2.melt(
    id_vars=None,
    value_vars=None,
    var_name=None,
    value_name='value',
    col_level=None,
) -> 'DataFrame'
Docstring:
Unpivot a DataFrame from wide to long format, optionally leaving identifiers set.

This function is useful to massage a DataFrame into a format where one
or more columns are identifier variables (`id_vars`), while all other
columns, considered measured variables (`value_vars`), are "unpivoted" to
the row axis, leaving just two non-identifier columns, 'variable' and
'value'.
.. versionadded:: 0.20.0

Parameters
----------
id_vars : tuple, list, or ndarray, optional
    Column(s) to use as identifier variables.
value_vars : tuple, list, or ndarray, optional
    Column(s) to unpivot. If not specified, uses all columns that
    are not set as `id_vars`.
var_name : scalar
    Name to use for the 'variable' column. If None it uses
    ``frame.columns.name`` or 'variable'.
value_name : scalar, default 'value'
    Name to use for the 'value'

In [6]:
stocks2.melt(id_vars='Date')

,Date,variable,value
0,2014-11-05,AAPL,108.860001
1,2014-11-06,AAPL,108.699997
2,2014-11-07,AAPL,109.010002
3,2014-11-10,AAPL,108.830002
4,2014-11-11,AAPL,109.699997
...,...,...,...
6280,2019-10-28,AMZN,1777.079956
6281,2019-10-29,AMZN,1762.709961
6282,2019-10-30,AMZN,1779.989990
6283,2019-10-31,AMZN,1776.660034
